## Célula 0: Instalar Dependências

In [5]:
# ==============================
# INSTALAÇÃO DE BIBLIOTECAS
# ==============================
import sys
!{sys.executable} -m pip install requests beautifulsoup4 pandas plotly jinja2 --quiet

print("✅ Bibliotecas instaladas! Reinicie o kernel se necessário (Kernel → Restart).")

✅ Bibliotecas instaladas! Reinicie o kernel se necessário (Kernel → Restart).



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importações e Configurações

In [6]:
# ==============================================================
# CGI.br Scraper - Versão Notebook (Independente)
# ==============================================================
# Coleta notícias e atas de reuniões do CGI.br
# Mostra dados na hora, gráficos interativos na célula, exporta CSV e banco SQLite local
# ==============================================================

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import sqlite3
import plotly.express as px
import plotly.io as pio
import time

# Gráficos interativos na célula
pio.renderers.default = "notebook_connected"

# Banco local
DATABASE_NAME = 'cgi_br_news.db'

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
}

print("✅ Pronto para scraping CGI.br!")

✅ Pronto para scraping CGI.br!


In [7]:
# ==============================
# BANCO DE DADOS SQLITE
# ==============================

DATABASE_NAME = 'internet_governance_news.db'

def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    
    conn.commit()
    conn.close()
    print("✅ Banco e tabela 'articles' prontos!")

create_database()


✅ Banco e tabela 'articles' prontos!


In [8]:
# ==============================
# FUNÇÃO DE INSERT
# ==============================

def insert_article(title, date, author, url, source):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, author, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, author, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()


## Scraper Notícias CGI.br

In [9]:
def scrape_cgi_noticias(max_pages=20):
    print("🔄 Scraping Notícias CGI.br...")
    start_time = time.time()
    articles = []
    base_url = "https://cgi.br/noticias/indice/"
    
    for page in range(1, max_pages + 1):
        url = base_url if page == 1 else f"https://cgi.br/noticias/indice/page:{page}"
        print(f"📄 Página {page}")
        
        response = requests.get(url, headers=HEADERS, timeout=30)
        if response.status_code == 404:
            break
        
        soup = BeautifulSoup(response.content, 'html.parser')
        news_items = soup.find_all('article', class_='row blog-post')
        if not news_items:
            break
        
        for item in news_items:
            title_tag = item.find('h2')
            title = title_tag.get_text(strip=True)
            link = "https://cgi.br" + title_tag.find('a')['href']
            
            date_tag = item.find('span', class_='date-post')
            date_str = date_tag.get_text(strip=True).upper()
            try:
                date = datetime.strptime(date_str, '%d %b %Y').strftime('%Y-%m-%d')
            except:
                date = date_str
            
            data = {
                'title': title,
                'date': date,
                'author': 'CGI.br Notícias',
                'url': link,
                'source': 'CGI Notícias'
            }
            
            articles.append(data)
            insert_article(**data)
        
        time.sleep(1.2)
    
    df = pd.DataFrame(articles)
    print(f"✅ {len(df)} notícias coletadas")
    display(df.head(10))
    print(f"⏱️ {time.time() - start_time:.2f}s")
    return df

df_noticias = scrape_cgi_noticias(max_pages=20)


🔄 Scraping Notícias CGI.br...
📄 Página 1
📄 Página 2
📄 Página 3
📄 Página 4
📄 Página 5
📄 Página 6
📄 Página 7
📄 Página 8
📄 Página 9
📄 Página 10
📄 Página 11
📄 Página 12
📄 Página 13
📄 Página 14
📄 Página 15
📄 Página 16
📄 Página 17
📄 Página 18
📄 Página 19
📄 Página 20
✅ 200 notícias coletadas


,title,date,author,url,source
0,"50 milhões de brasileiros já usam IA, mas pote...",09 DEZ 2025,CGI.br Notícias,https://cgi.br/noticia/releases/50-milhoes-de-...,CGI Notícias
1,15ª Semana de Infraestrutura da Internet debat...,09 DEZ 2025,CGI.br Notícias,https://cgi.br/noticia/releases/15-semana-de-i...,CGI Notícias
2,NIC.br celebra 20 anos de atuação pelo desenvo...,05 DEZ 2025,CGI.br Notícias,https://cgi.br/noticia/releases/nic-br-celebra...,CGI Notícias
3,No Dia Internacional da Pessoa com Deficiência...,03 DEZ 2025,CGI.br Notícias,https://cgi.br/noticia/releases/no-dia-interna...,CGI Notícias
4,Estudo do Cetic.br revela demanda por formação...,2025-11-25,CGI.br Notícias,https://cgi.br/noticia/releases/estudo-do-ceti...,CGI Notícias
5,CGI.br e NIC.br lançam programa de incentivo à...,2025-11-11,CGI.br Notícias,https://cgi.br/noticia/releases/cgi-br-e-nic-b...,CGI Notícias
6,"Nota de pesar: Rubens Kühl, gerente de produto...",2025-11-03,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-pesar-rub...,CGI Notícias
7,"Note of Condolence: Rubens Kühl, Registro.br P...",2025-11-03,CGI.br Notícias,https://cgi.br/noticia/notas/note-of-condolenc...,CGI Notícias
8,TIC Kids Online Brasil: 65% das crianças e dos...,22 OUT 2025,CGI.br Notícias,https://cgi.br/noticia/releases/tic-kids-onlin...,CGI Notícias
9,10º Simpósio de Crianças e Adolescentes na Int...,16 OUT 2025,CGI.br Notícias,https://cgi.br/noticia/releases/10-simposio-de...,CGI Notícias


⏱️ 35.75s


## Scraper Reuniões CGI.br

In [10]:
def scrape_cgi_reunioes():
    print("🔄 Scraping Reuniões CGI.br...")
    url = "https://www.cgi.br/reunioes/extraordinarias/"
    response = requests.get(url, headers=HEADERS, timeout=20)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    date_pattern = re.compile(r'\d{2} de [a-zç]+ de \d{4}', re.I)
    date_elements = soup.find_all(string=date_pattern)
    
    articles = []
    for date_elem in date_elements:
        date = date_elem.strip()
        parent = date_elem.find_parent('div')
        ul = parent.find_next('ul') if parent else None
        if not ul:
            continue
        
        for item in ul.find_all('li'):
            title = item.get_text(strip=True)
            link_tag = item.find('a')
            link = "https://www.cgi.br" + link_tag['href'] if link_tag else url
            
            data = {
                'title': title,
                'date': date,
                'author': 'CGI.br Reuniões',
                'url': link,
                'source': 'CGI Reuniões'
            }
            
            articles.append(data)
            insert_article(**data)
    
    df = pd.DataFrame(articles)
    print(f"✅ {len(df)} atas coletadas")
    display(df.head(10))
    return df

df_reunioes = scrape_cgi_reunioes()


🔄 Scraping Reuniões CGI.br...
✅ 30 atas coletadas


,title,date,author,url,source
0,Política de Privacidade do WhatsApp,04 de agosto de 2025,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
1,Política de Privacidade do WhatsApp,13 de setembro de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
2,Política de Privacidade do WhatsApp,09 de julho de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
3,Visite o site do CGI.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.brhttps://www.cgi.br,CGI Reuniões
4,Visite o site do NIC.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.brhttps://www.nic.br,CGI Reuniões
5,Visite o site do Registro.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.brhttps://www.registro.br,CGI Reuniões
6,Visite o site do CERT.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.brhttps://www.cert.br,CGI Reuniões
7,Visite o site do CETIC.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.brhttps://www.cetic.br,CGI Reuniões
8,Visite o site do CEPTRO.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.brhttps://www.ceptro.br,CGI Reuniões
9,Visite o site do Ceweb,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.brhttps://www.ceweb.br/,CGI Reuniões


In [11]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("SELECT * FROM articles ORDER BY date DESC", conn)
    conn.close()
    return df

df_db = load_articles()
display(df_db.head(20))
print(f"📦 Total no banco: {len(df_db)} registros")


,id,title,date,author,url,source
0,74,"Em duas décadas, proporção de lares urbanos br...",31 OUT 2024,CGI.br Notícias,https://cgi.br/noticia/releases/em-duas-decada...,CGI Notícias
1,188,"No FIB13, especialistas apontam expectativas e...",31 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/no-fib-13-espe...,CGI Notícias
2,12,Mais acessível e completo: conheça o novo site...,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/notas/mais-acessivel-e-...,CGI Notícias
3,13,Evento em São Paulo celebra 15 anos de SACI-Ad...,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/releases/evento-em-sao-...,CGI Notícias
4,189,Abertas as inscrições para o 13º Fórum da Inte...,30 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/abertas-as-ins...,CGI Notícias
5,119,Declaração final do NETmundial+10 apresenta re...,30 ABR 2024,CGI.br Notícias,https://cgi.br/noticia/releases/declaracao-fin...,CGI Notícias
6,83,Nota de falecimento: Mariana Stanton,29 SET 2024,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-falecimen...,CGI Notícias
7,159,Nota de falecimento: Luiz Gonzaga Lauschner,29 SET 2023,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-falecimen...,CGI Notícias
8,38,Fórum da Internet no Brasil resgata legado da ...,29 MAI 2025,CGI.br Notícias,https://cgi.br/noticia/releases/forum-da-inter...,CGI Notícias
9,39,Prêmio Destaques homenageia 12 personalidades ...,29 MAI 2025,CGI.br Notícias,https://cgi.br/noticia/releases/premio-destaqu...,CGI Notícias


📦 Total no banco: 210 registros


## Combinar Dados + Filtro + Gráficos

In [13]:
import re
import plotly.express as px
import pandas as pd

def plot_charts(df):
    if df.empty:
        print("❌ Sem dados para plotar.")
        return

    # ------------------------------
    # Top 15 Notícias
    # ------------------------------
    top15 = df.head(15).copy()
    top15['rank'] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x='rank',
        y='title',
        orientation='h',
        title='Top 15 Notícias – Internet Governance'
    )

    fig1.update_layout(
        height=600,
        yaxis={'categoryorder': 'array', 'categoryarray': top15['title'][::-1]}
    )
    fig1.show()

    # ------------------------------
    # Distribuição por Fonte
    # ------------------------------
    source_count = df['source'].value_counts().reset_index()
    source_count.columns = ['source', 'count']

    fig2 = px.pie(
        source_count,
        names='source',
        values='count',
        title='Distribuição de Notícias por Fonte'
    )
    fig2.show()

    # ------------------------------
    # Nuvem de Palavras
    # ------------------------------
    text = ' '.join(df['title'].astype(str)).lower()
    words = re.findall(r'\b\w{4,}\b', text)

    if not words:
        print("⚠️ Texto insuficiente para nuvem de palavras.")
        return

    word_freq = (
        pd.Series(words)
        .value_counts()
        .head(20)
        .reset_index()
    )
    word_freq.columns = ['palavra', 'freq']

    fig3 = px.treemap(
        word_freq,
        path=['palavra'],
        values='freq',
        title='Nuvem de Palavras – Títulos'
    )
    fig3.show()


In [14]:
plot_charts(df_db)